Perhaps mark each pdf with a code denoting the bank type, so this file only has to do a simple string parse to know what to do. i.e. 'JohnDoeCAP.pdf', 'MarkAnthonyNED.pdf', etc. 

Sometimes Description is long and overflows. I cut it off, but should we keep it all?

Formulae:
Number of transactions per month/amount of customers
Total money in/out subtract out-in for net
Bin description by types/prevalence 

In [1]:
import math 
import tabula as tb
from tabula import read_pdf

#constant conversion factor, per adobe
#pdf measurer pdf unit = 1/72 in
cf =72  

#all measurements in format [top,left,bottom,width]

#page counter
from PyPDF2 import PdfFileReader
def count_pdf_pages(file):
    with open(file, 'rb') as pdf:
        page_count = PdfFileReader(pdf).getNumPages()
    return page_count

In [2]:
import pandas as pd
#CAPITEC
#complete

def CAP_scan(filename):
    #capitec measurements
    #first page:
    page_1_box = [3.77, 0.4, 9.8, 8.5,]
    #middle pages:
    middle_pages_box = [0.3, 0.4, 10.2, 8.5]

  
    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box)):
        page_1_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page
    page_1_table = tb.read_pdf(filename, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
    page_1_df = page_1_table[0].drop(labels='Posting Date', axis=1)

    #clean first page
    page_1_df = page_1_df.rename(columns={'Transaction': 'Transaction Date'})
    page_1_df = page_1_df.drop(index=0)
    page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

    #append first page
    total_df = pd.DataFrame(columns=page_1_df.columns)
    total_df = pd.concat([total_df, page_1_df])


    for page in range(2, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]

        #clean middle page
        middle_df = middle_df.dropna(axis='columns', how='all')
        middle_df.columns = page_1_df.columns


        middle_df = middle_df.dropna(subset=['Transaction Date','Balance (R)'])
        middle_df['Transaction Date'] = middle_df['Transaction Date'].map(lambda x: str(x).split()[-1])

        #append to total df
        total_df = pd.concat([total_df, middle_df])

        
    #combine money in and money out columns
    total_df = total_df.bfill(axis=1)
    
    total_df = total_df.drop(['Money Out (R)'], axis=1)
    
    total_df = total_df.rename(columns={'Money In (R)': 'Amount'})
    return total_df




In [4]:
import pandas as pd
#NED Bank



#mercantile measurements
#first page:
page_1_box = [4.2, 0.4, 8.9, 7.7,]
#middle pages:
middle_pages_box = [2.4, 0.4, 10.1, 7.7]

page_count = count_pdf_pages(fileaname)

#convert boxes
for i in range(0, len(page_1_box)):
    page_1_box[i] *= cf
    
for i in range(0, len(middle_pages_box)):
    middle_pages_box[i] *= cf

#get first page
page_1_table = tb.read_pdf(file, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
page_1_df = page_1_table[0].drop(labels='Post', axis=1)

#page_1_df
#clean first page
page_1_df = page_1_df.rename(columns={'Trans.': 'Transaction Date'})
page_1_df = page_1_df.drop(index=0)
######page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance (R)'])

#append first page
total_df = pd.DataFrame(columns=page_1_df.columns)
total_df = pd.concat([total_df, page_1_df])


for page in range(2, page_count+1):

    #get middle page
    middle_pages_table = tb.read_pdf(file, pages=page, area=[middle_pages_box],
                                 output_format="dataframe", stream=True, pandas_options={'header': None})
    middle_df = middle_pages_table[0]

    #clean middle page
    middle_df = middle_df.drop(labels=0, axis=1)
    middle_df = middle_df.drop(index=[0,1])
    middle_df = middle_df.dropna(axis='columns', how='all')
    middle_df.columns = page_1_df.columns
    middle_df = middle_df.dropna(subset=['Transaction Date','Balance'])

    #append to total df
    total_df = pd.concat([total_df, middle_df])





total_df


NameError: name 'file' is not defined

In [60]:
import pandas as pd
#Mercantile Bank
#can these be longer than the one provided??

    
def MER_scan(filename):

    #mercantile measurements
    #first page:
    page_1_box = [4.2, 0.4, 8.9, 7.7,]
    #middle pages:
    middle_pages_box = [2.4, 0.4, 10.1, 7.7]

    page_count = count_pdf_pages(filename)

    #convert boxes
    for i in range(0, len(page_1_box)):
        page_1_box[i] *= cf

    for i in range(0, len(middle_pages_box)):
        middle_pages_box[i] *= cf

    #get first page
    page_1_table = tb.read_pdf(filename, pages=1,area=[page_1_box],output_format="dataframe", stream=True)
    page_1_df = page_1_table[0].drop(labels='Post', axis=1)

    #page_1_df
    #clean first page
    page_1_df = page_1_df.rename(columns={'Trans.': 'Transaction Date'})
    page_1_df = page_1_df.drop(index=0)
    page_1_df = page_1_df.dropna(subset=['Transaction Date','Balance'])

    #append first page
    total_df = pd.DataFrame(columns=page_1_df.columns)
    total_df = pd.concat([total_df, page_1_df])


    for page in range(2, page_count+1):

        #get middle page
        middle_pages_table = tb.read_pdf(filename, pages=page, area=[middle_pages_box],
                                     output_format="dataframe", stream=True, pandas_options={'header': None})
        middle_df = middle_pages_table[0]

        #clean middle page
        middle_df = middle_df.drop(labels=0, axis=1)
        middle_df = middle_df.drop(index=[0,1])
        middle_df = middle_df.dropna(axis='columns', how='all')
        middle_df.columns = page_1_df.columns
        middle_df = middle_df.dropna(subset=['Transaction Date','Balance'])

        #append to total df
        total_df = pd.concat([total_df, middle_df])

        
    #drop reference
    total_df = total_df.drop(['Reference'], axis=1)
    
    #turn nan charges into 0 
    total_df['Fees'] = total_df['Fees'].fillna(0)
    
    #numerize fees, amount, and balance
  

    total_df["Fees"] = total_df["Fees"].replace(' ', '', regex=True).astype(float)
    total_df["Amount"] = total_df["Amount"].replace(' ', '', regex=True).astype(float)
    total_df["Balance"] = total_df["Balance"].replace(' ', '', regex=True).astype(float)


    #combine fees and amount and drop fees
    total_df["Amount"] = total_df["Fees"] + total_df["Amount"]
    total_df = total_df.drop(["Fees"], axis=1)

    return total_df


In [61]:
filename = "#MER#Mercantile Bank.pdf"
df = MER_scan(filename)
#elif filename.startswith("#FNB#"):
#    df = FNB_scan(filename)
#elif filename.startswith("#NED#"):
#    df = NED_scan(filename)
#elif filename.startswith("#MER#"):
#    df = MER_scan(filename)

df

,Transaction Date,Description,Fees,Amount,Balance
3,06/01/22,POS Local Purchase,-205.00,-200.00,2115.16
5,07/01/22,POS Local Purchase,-86.00,-81.00,2029.16
7,07/01/22,ATM Withdrawal,-1009.75,-1000.00,1019.41
8,07/01/22,POS Local Purchase,-186.30,-181.30,833.11
10,07/01/22,Fuel Purchase,-105.00,-100.00,728.11
12,07/01/22,POS Local Purchase,-121.00,-116.00,607.11
14,08/01/22,Inward EFT Credit,20000.00,20000.00,20607.11
15,08/01/22,******006073****,-15927.50,-15920.00,4679.61
16,08/01/22,Fuel Purchase,-205.00,-200.00,4474.61
18,08/01/22,POS Local Purchase,-249.95,-244.95,4224.66


In [33]:
df["Amount"].iloc[0]

'-200.00'

In [ ]:
filename = input("enter name of file to be scanned")
if filename.startswith("#CAP#"):
    df = CAP_scan(filename)
elif filename.startswith("#FNB#"):
    df = FNB_scan(filename)
elif filename.startswith("#NED#"):
    df = NED_scan(filename)
elif filename.startswith("#MER#"):
    df = MER_scan(filename)
print(df)


In [9]:
import pandas as pd

df1 = pd.DataFrame([['a', 1,2,3,4], ['b', 2],['a', 1], ['b', 2],['a', 1]],
                   columns=range(0,5))
df1

,0,1,2,3,4
0,a,1,2.0,3.0,4.0
1,b,2,NaN,NaN,NaN
2,a,1,NaN,NaN,NaN
3,b,2,NaN,NaN,NaN
4,a,1,NaN,NaN,NaN
